In [ ]:
import tensorflow as tf
from scipy.misc import imread
import numpy as np
import os
from PIL import Image
import itertools
import time

In [ ]:
DATA_DIR = os.path.join(os.path.pardir, "dataset")
assert os.path.exists(DATA_DIR)
TRAINING_SET_SIZE = 400
BATCH_SIZE = 10
IMAGE_SIZE = 1536
N_CHANNEL = 3
N_CLASSES = 4
FLAT_LEN = IMAGE_SIZE**2

FEATURES_LIST = {"image/encoded": tf.FixedLenFeature([], tf.string),
        "image/height": tf.FixedLenFeature([], tf.int64),
        "image/width": tf.FixedLenFeature([], tf.int64),
        "image/filename": tf.FixedLenFeature([], tf.string),
        "image/class/label": tf.FixedLenFeature([], tf.int64),}

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

class _image_object:
    def __init__(self):
        self.image = tf.Variable([], dtype = tf.string)
        self.height = tf.Variable([], dtype = tf.int64)
        self.width = tf.Variable([], dtype = tf.int64)
        self.filename = tf.Variable([], dtype = tf.string)
        self.label = tf.Variable([], dtype = tf.int32)

def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, features=FEATURES_LIST)
    image_encoded = features["image/encoded"]
    image_raw = tf.image.decode_jpeg(image_encoded, channels=N_CHANNEL)
    image_object = _image_object()
    image_object.image = tf.image.resize_image_with_crop_or_pad(image_raw, IMAGE_SIZE, IMAGE_SIZE)
    image_object.height = features["image/height"]
    image_object.width = features["image/width"]
    image_object.filename = features["image/filename"]
    image_object.label = tf.cast(features["image/class/label"], tf.int64)
    return image_object

In [5]:
classes = ["benign_jpg", "insitu_jpg", "invasive_jpg", "normal_jpg"]
dataset_path = os.path.join(os.path.pardir, 'dataset')
N_c = 100
N = N_c*4
size = 1536
rgb_channel = 3

dataset = np.ndarray([N, size, size, rgb_channel])

In [6]:
'''
start_time = time.time()
for i, class_name in enumerate(classes):
    path = os.path.join(dataset_path, class_name)
    files = os.listdir(path)
    for j, img_name in enumerate(files):
        if j < N_c:
            im = Image.open(path + "/" + img_name)
            imarray = np.array(im)
            dataset[i*N_c + j] = imarray
    print(path + "/*", "... loaded")
end_time = time.time()
run_time = end_time-start_time
print(run_time)
'''

../dataset/benign_jpg/* ... loaded
../dataset/insitu_jpg/* ... loaded
../dataset/invasive_jpg/* ... loaded
../dataset/normal_jpg/* ... loaded
33.33758854866028


In [18]:
'''
start_time = time.time()
np.save("dataset.npy", dataset)
end_time = time.time()
run_time = end_time-start_time
print(run_time)

start_time = time.time()
dataset = np.load("dataset.npy")
end_time = time.time()
run_time = end_time-start_time
print(run_time)
'''

'\nstart_time = time.time()\nnp.save("dataset.npy", dataset)\nend_time = time.time()\nrun_time = end_time-start_time\nprint(run_time)\n\nstart_time = time.time()\ndataset = np.load("dataset.npy")\nend_time = time.time()\nrun_time = end_time-start_time\nprint(run_time)\n'

In [ ]:
images_name =
for i, class_name in enumerate(classes):
    path = os.path.join(dataset_path, class_name)
    files = os.listdir(path)
    for j, img_name in enumerate(files):
        if j < N_c:
            im = Image.open(path + "/" + img_name)
            imarray = np.array(im)
            dataset[i*N_c + j] = imarray

In [7]:
#np.random.shuffle(dataset)
index_permutation = np.array(range(N))
np.random.shuffle(index_permutation)
print(index_permutation)

[253 336 136 254 362 216 181 116 310  48  96 144 245 382 118 280 201 121
  45 322 137 241 111 363 146 328 166 298   7 120 316 264  25 353 374  30
  16 290 207 187 289 309  13 205 170 277 124  64 153  78 214 397 341 203
 210 211   5  24 369 265 291 131  89  71 178 173 156 302 213 145  84  39
 350 389 266  56 331 379   6 394 175  83  32 301 123 246 209  57 113 269
 368 242 140 358 388 168 108  92  54  53 251 225 334 337 324  80 356 366
  18 395 232 335 165 255  65 192 268 303  44 311 189 295 380 355 385 194
 204  20 372 227 218 104 352   4 315 162   3 234 160 133 338 249 292  35
 378  76 176  47  67 377 344 357 329 138 197 330 125 312 283 193  94 278
  90 101 196 155 150 198  63 177 223 167 247 231  21 217 183  95  58 149
  11 318 110 279 157  19 296 287  43 215  93  74 262 271 305 135 383 202
 117 257 161 297 348 392  61 263  37 299 235 164 233  87 129 172 273 326
 244 151 339 258 387 143 182  26 239 261 288 154 396  72 186 171  77  29
 367   9 107  17 323   8 346  49  55 313 360 184  7

In [8]:
n_training = int(.8 * N)
n_validation = int(.1 * N)
n_test = int(.1 * N)

training_set = dataset[index_permutation[0:n_training]]
validation_set = dataset[index_permutation[n_training:n_training+n_validation]]
test_set = dataset[index_permutation[n_training+n_validation:n_training+n_validation+n_test]]

print(training_set.shape)
print(validation_set.shape)
print(test_set.shape)

MemoryError: 